In [ ]:
# ! pip install mysql-connector-python


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd

# Read the csv file

df = pd.read_csv("traffic_details.csv")
df.head()

C:\Users\Hemgiri\AppData\Local\Temp\ipykernel_12600\3482217159.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("traffic_details.csv")


,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305


In [7]:
# To drop the driver_race column from original dataframe(df)
# df.drop(columns = "driver_race", inplace = True)
df.head()

,stop_date,stop_time,country_name,driver_gender,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305


In [2]:
# To drop the driver_age_raw column from original dataframe(df)
df.drop(columns = "driver_age_raw", inplace = True)

In [4]:
# list the data types

df.dtypes

stop_date             object
stop_time             object
country_name          object
driver_gender         object
driver_age             int64
violation_raw         object
violation             object
search_conducted        bool
search_type           object
stop_outcome          object
is_arrested             bool
stop_duration         object
drugs_related_stop      bool
vehicle_number        object
dtype: object

In [5]:
# find the sum of NAN values

df.isna().sum()

stop_date                 0
stop_time                 0
country_name              0
driver_gender             0
driver_age                0
violation_raw             0
violation                 0
search_conducted          0
search_type           21720
stop_outcome              0
is_arrested               0
stop_duration             0
drugs_related_stop        0
vehicle_number            0
dtype: int64

In [3]:
# NAN values handled by using forward fill method 

df = df.fillna(method='ffill')
df.isna().sum()

C:\Users\Hemgiri\AppData\Local\Temp\ipykernel_12600\929637020.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


stop_date             0
stop_time             0
country_name          0
driver_gender         0
driver_age            0
driver_race           0
violation_raw         0
violation             0
search_conducted      0
search_type           0
stop_outcome          0
is_arrested           0
stop_duration         0
drugs_related_stop    0
vehicle_number        0
dtype: int64

In [4]:
# convert an object to datetime

df["stop_date"] = pd.to_datetime(df["stop_date"])

In [14]:
# Establising connection btw mysql and python

import mysql.connector

mydb = mysql.connector.connect(
    host = "localhost",
    username = "root",
    password = "",
    port = 3307

)

mycursor = mydb.cursor(buffered=True)

In [15]:
# # Create a database 

# mycursor.execute("CREATE DATABASE digital_ledger")

mycursor.execute("USE digital_ledger")

In [ ]:
# create a table

mycursor.execute("CREATE TABLE traffic_records(ID INT(10) PRIMARY KEY AUTO_INCREMENT, stop_date DATE, stop_time TIME, country_name VARCHAR(100), driver_gender CHAR(1), driver_age INT(2), driver_race VARCHAR(255), violation_raw VARCHAR(255), violation VARCHAR(255), search_conducted BOOLEAN, search_type VARCHAR(255), stop_outcome VARCHAR(20), is_arrested BOOLEAN, stop_duration VARCHAR(20), drugs_related_stop BOOLEAN, vehicle_number CHAR(10))")

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'EXIST traffic_records(ID INT(10) PRIMARY KEY AUTO_INCREMENT, stop_date DATE, ...' at line 1

In [17]:
# Insert the df vaules in sql

for index, row in df.iterrows():
    mycursor.execute("""
        INSERT INTO traffic_records(stop_date, stop_time, country_name, driver_gender, driver_age, driver_race, violation_raw, violation, search_conducted, search_type, stop_outcome, is_arrested, stop_duration, drugs_related_stop, vehicle_number)
        VALUES (%s, %s, %s, %s, %s, %s,  %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, tuple(row))
mydb.commit()
print("Data inserted completed")

Data inserted completed
